In [2]:
import moon.data
import moon.problem

In [3]:
probs = moon.data.read_problems('data/cleaned_probs.csv')

In [24]:
import numpy as np

x = np.array([p.array for p in probs])
y = np.array([p.grade.onehot for p in probs])

print(x.shape, y.shape)

split = 0.2
indices = np.arange(len(probs))
np.random.shuffle(indices)
n_test = int(len(probs) * split)

x_train, y_train = x[indices[n_test:]], y[indices[n_test:]]
x_test, y_test = x[indices[0:n_test]], y[indices[0:n_test]]
print((x_train.shape, y_train.shape), (x_test.shape, y_test.shape))

(30991, 18, 11) (30991, 17)
((24793, 18, 11), (24793, 17)) ((6198, 18, 11), (6198, 17))


In [47]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K


# Metric to compute "accuracy within k" ordinal classes, if the output is independent sigmoids.
# See section 2.3: http://calla.rnet.missouri.edu/cheng_courses/rank.pdf
# Written but no longer used; instead we use a weighted cross-entropy for loss and softmax (one-hot) for output.
class OrdinalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='acc_within_k', k=0, threshold=0.5, **kwargs):
        super(OrdinalAccuracy, self).__init__(name=name, **kwargs)
        self.k = k
        self.threshold = threshold
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        pred_thresh = tf.cast(y_pred >= self.threshold, 'int8')
        true_thresh = tf.cast(y_true >= self.threshold, 'int8')
        # Since tf.argmin returns the lowest index possible, this gets us the index of the last True value.
        # Note that this is -1 if the network doesn't many any prediction at all.
        pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
        true_idx = tf.argmin(true_thresh, axis=-1) - 1
        distance_bools = (tf.abs(pred_idx - true_idx) <= self.k)
        correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
        incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
        self.total.assign_add(correct)
        self.count.assign_add(correct + incorrect)
    
    def result(self):
        return self.total / self.count


# Metric to compute "accuracy within k" ordinal classes, if the output is softmax.
class OrdinalAccuracySoftmax(tf.keras.metrics.Metric):
    def __init__(self, name='acc_within_k', k=0, **kwargs):
        super(OrdinalAccuracySoftmax, self).__init__(name=name, **kwargs)
        self.k = k
        self.correct = self.add_weight(name='correct', initializer='zeros')
        self.incorrect = self.add_weight(name='incorrect', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        pred_idx = tf.argmax(y_pred, axis=-1)
        true_idx = tf.argmax(y_true, axis=-1)
        distance_bools = (tf.abs(pred_idx - true_idx) <= self.k)
        correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
        incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
        self.correct.assign_add(correct)
        self.incorrect.assign_add(incorrect)     
    
    def result(self):
        return self.correct / (self.correct + self.incorrect)


# https://github.com/JHart96/keras_ordinal_categorical_crossentropy GPLv3 license
def ordinal_categorical_crossentropy_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * tf.keras.losses.categorical_crossentropy(y_true, y_pred)

# see
# https://arxiv.org/abs/1901.07884 and https://github.com/Raschka-research-group/coral-cnn/issues/9
# more background https://arxiv.org/abs/1705.05278

In [50]:
# α = 0.3
p = .4
input_shape = moon.problem.Problem.GRID_SHAPE
hiddens = [16, 16]
hidden_activation = 'swish'
output_shape = moon.problem.Grade.N_GRADES
output_activation = 'softmax'

metrics = [OrdinalAccuracySoftmax(name='acc0', k=0), OrdinalAccuracySoftmax(name='acc1', k=1), OrdinalAccuracySoftmax(name='acc2', k=2)]
loss = ordinal_categorical_crossentropy_loss
# loss = 'categorical_crossentropy'
adam_lr = 1e-3
optim = tf.keras.optimizers.Adam(lr=adam_lr)

in_x = layers.Input(shape=input_shape)
features = layers.Flatten()(in_x)
for nodes in hiddens:
    features = layers.Dense(nodes, activation=hidden_activation)(features)
    # features = layers.ReLU(α)(features)
    if p > 0: features = layers.Dropout(p)(features)
out_y = layers.Dense(output_shape, activation=output_activation)(features)

model = tf.keras.Model(in_x, out_y, name='Feedforward_classifier')
model.compile(loss=loss, optimizer=optim, metrics=metrics)
# model.summary()

In [51]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=4, verbose=1)]
batch_size = 64
max_epochs = 300

history = model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=max_epochs, callbacks=callbacks)

# [256, 128, 64], p=.4, flattened, 28e
# loss: 0.0396 - acc0: 0.4144 - acc1: 0.7597 - acc2: 0.9229 - val_loss: 0.0414 - val_acc0: 0.3925 - val_acc1: 0.7375 - val_acc2: 0.9140

# [16, 16], p=.4, flattened, 76e
# loss: 0.0446 - acc0: 0.3766 - acc1: 0.7139 - acc2: 0.8944 - val_loss: 0.0416 - val_acc0: 0.3879 - val_acc1: 0.7438 - val_acc2: 0.9138

# [16, 16], p=.4, flattened, onehots!, 71e, softmax+cce
# loss: 1.5533 - acc0: 0.4083 - acc1: 0.6546 - acc2: 0.8327 - val_loss: 1.4997 - val_acc0: 0.4226 - val_acc1: 0.6822 - val_acc2: 0.8567
# [16, 16], p=.4, flattened, onehots, 49e, softmax+ordinalcce
# loss: 1.7202 - acc0: 0.4096 - acc1: 0.6641 - acc2: 0.8420 - val_loss: 1.6567 - val_acc0: 0.4193 - val_acc1: 0.6813 - val_acc2: 0.8616

# [16, 16], p=.4, flattened, lr 1e-4, 202e
# loss: 0.0463 - acc0: 0.3726 - acc1: 0.6968 - acc2: 0.8863 - val_loss: 0.0429 - val_acc0: 0.3987 - val_acc1: 0.7257 - val_acc2: 0.9037

# [32], p=.4, flattened, 31e
# loss: 0.0431 - acc0: 0.3896 - acc1: 0.7303 - acc2: 0.9021 - val_loss: 0.0416 - val_acc0: 0.4055 - val_acc1: 0.7385 - val_acc2: 0.9096

# [256, 128, 64], p=.4, 3d, 24e
# loss: 0.0409 - acc0: 0.4075 - acc1: 0.7475 - acc2: 0.9123 - val_loss: 0.0421 - val_acc0: 0.3945 - val_acc1: 0.7389 - val_acc2: 0.9076

# [64, 64], p=.4, 3d, 32e
# loss: 0.0420 - acc0: 0.4018 - acc1: 0.7372 - acc2: 0.9087 - val_loss: 0.0419 - val_acc0: 0.3992 - val_acc1: 0.7396 - val_acc2: 0.9080

# [64], p=.4, 3d, 44e
# loss: 0.0405 - acc0: 0.4103 - acc1: 0.7505 - acc2: 0.9152 - val_loss: 0.0416 - val_acc0: 0.4016 - val_acc1: 0.7402 - val_acc2: 0.9092

# [32], p=.4, 3d, 59e
# loss: 0.0418 - acc0: 0.4037 - acc1: 0.7396 - acc2: 0.9084 - val_loss: 0.0420 - val_acc0: 0.4045 - val_acc1: 0.7314 - val_acc2: 0.9093

Epoch 1/300
388/388 [==============================] - 1s 3ms/step - loss: 2.4705 - acc0: 0.3091 - acc1: 0.4770 - acc2: 0.6432 - val_loss: 1.9506 - val_acc0: 0.3675 - val_acc1: 0.5544 - val_acc2: 0.7191
Epoch 2/300
388/388 [==============================] - 1s 3ms/step - loss: 1.9909 - acc0: 0.3646 - acc1: 0.5730 - acc2: 0.7456 - val_loss: 1.8308 - val_acc0: 0.3871 - val_acc1: 0.6086 - val_acc2: 0.7815
Epoch 3/300
388/388 [==============================] - 1s 3ms/step - loss: 1.9195 - acc0: 0.3734 - acc1: 0.5966 - acc2: 0.7698 - val_loss: 1.7903 - val_acc0: 0.3922 - val_acc1: 0.6273 - val_acc2: 0.7959
Epoch 4/300
388/388 [==============================] - 1s 3ms/step - loss: 1.8709 - acc0: 0.3800 - acc1: 0.6040 - acc2: 0.7845 - val_loss: 1.7623 - val_acc0: 0.3985 - val_acc1: 0.6397 - val_acc2: 0.8098
Epoch 5/300
388/388 [==============================] - 1s 3ms/step - loss: 1.8487 - acc0: 0.3838 - acc1: 0.6163 - acc2: 0.7955 - val_loss: 1.7495 - val_acc0: 0.4008 - val_acc1: 0.6434 - va

388/388 [==============================] - 1s 2ms/step - loss: 1.7261 - acc0: 0.4081 - acc1: 0.6576 - acc2: 0.8382 - val_loss: 1.6551 - val_acc0: 0.4195 - val_acc1: 0.6859 - val_acc2: 0.8617
Epoch 42/300
388/388 [==============================] - 1s 2ms/step - loss: 1.7202 - acc0: 0.4096 - acc1: 0.6641 - acc2: 0.8420 - val_loss: 1.6567 - val_acc0: 0.4193 - val_acc1: 0.6813 - val_acc2: 0.8616
Epoch 00042: early stopping


In [78]:
# print(np.round(model.predict(x_test[99][np.newaxis, :]) * 100))
# print(y_test[99])
y_true = y_test[[2, 4, 8, 16, 32]]
y_pred = model.predict(x_test[[2, 4, 8, 16, 32]])

In [79]:
pred_thresh = tf.cast(y_pred >= .5, 'int8')
true_thresh = tf.cast(y_true >= .5, 'int8')
print(pred_thresh, true_thresh)

# Since tf.argmin returns the lowest index possible, this gets us the index of the last True value.
# Note that this is -1 if the network doesn't many any prediction at all.
pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
true_idx = tf.argmin(true_thresh, axis=-1) - 1
print(pred_idx, true_idx)

distance_bools = tf.abs(pred_idx - true_idx) <= 0
correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
print(distance_bools, correct, incorrect)

print(correct / (correct+incorrect))

tf.Tensor(
[[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8) tf.Tensor(
[[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8)
tf.Tensor([7 8 7 8 6], shape=(5,), dtype=int64) tf.Tensor([8 9 6 7 6], shape=(5,), dtype=int64)
tf.Tensor([False False False False  True], shape=(5,), dtype=bool) tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(0.2, shape=(), dtype=float32)
